# Init

In [3]:
from utils import *
from SuperGlue import *
p=11
ks=50
Use_SIFT=False
burst_path = '../images/bookshelf'
file_extension='*.jpg'
gaussian_ksize=31 # gaussian kernel size
burst=read_burst(burst_path, file_extension)

# Bursr Registration
reference source: `https://learnopencv.com/image-alignment-feature-based-using-opencv-c-python/`

In [4]:
# Use the clearest image as the reference image for burst registration
a = np.fft.fft2(np.moveaxis(burst, 3, 1))
a = np.sum(np.abs(a), axis=(1, 2, 3))
a = np.argmax(a)
burst[[0, a], :, :, :] = burst[[a, 0], :, :, :]

In [5]:
if Use_SIFT:
    burst=register_burst(burst)
else:
    burst=register_burst_SuperGlue(burst, copy=False, force_cpu=False)

Running inference on device "cuda"
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)
process:1/10
process:2/10
process:3/10
process:4/10
process:5/10
process:6/10
process:7/10
process:8/10
process:9/10


In [6]:
# cut the edge in order to remove black pixels
burst=burst[:,20:-20, 20:-20,:]

In [7]:
# visualize warp result
print(burst.shape)
plt.figure(figsize=(100,80))
for i in range(burst.shape[0]):
    plt.subplot(math.ceil(burst.shape[0]/3),3,i+1)
    plt.imshow(cv2.cvtColor(burst[i],cv2. COLOR_BGR2RGB))

if Use_SIFT:
    plt.show()
else:
    plt.savefig('./warp_result.png')

(10, 2161, 3461, 3)


# FBA

In [8]:
# np.moveaxis: change the shape from (# of img, R, C, color) to (# of img, color, R, C)
spectrums=np.fft.fft2(np.moveaxis(burst, 3, 1))
# spectrum.shape = (# of img, color, R, C)

# get the spectrum of a blur kernel
shape=spectrums.shape[-2:]
sig=min(shape)/ks
# blur_kernel_spectrum=get_gau_ker(gaussian_ksize, sig, shape)[1]

# average color channels
weight=np.mean(np.abs(spectrums), axis=1)

# pass through the gaussian filter
# weight=weight*blur_kernel_spectrum
weight=np.fft.fftshift(weight)
for i in range(weight.shape[0]):
        weight[i]=cv2.GaussianBlur(weight[i,:,:], (31,31), sig)
weight=np.fft.ifftshift(weight)

weight=np.power(weight, p)
weight/=np.sum(weight, axis=0)

# expand the shape of the weight from (# of img, R, C) to (# of img, color, R, C)
weight=np.repeat(np.expand_dims(weight, axis=1), 3, axis=1)

# restore image
spectrum_restored=np.sum(weight*spectrums, axis=0)
image_restored=np.fft.ifft2(spectrum_restored)

# change the shape from (color, R, C) to (R, C, color)
image_restored=np.moveaxis(image_restored, 0, 2)

# restore to uint8
image_restored=image_restored.real
print(f'# of pixels less than 0: {np.sum(image_restored<0)}\n# of pixels more than 255: {np.sum(image_restored>255)}')
image_restored=np.where(image_restored<0,0,image_restored)
image_restored=np.where(image_restored>255,255,image_restored).astype(np.uint8)

# of pixels less than 0: 0
# of pixels more than 255: 357794


In [9]:
t=np.fft.ifft2(spectrum_restored)
b=np.moveaxis(t.real, 0, 2)
b=b-b.min()
b=(b/b.max()*255).astype(np.uint8)

```
第一張圖的weight在邊緣會比較高
所以是不是要盡力挑第一張圖，也就是別人對齊的參考圖?
```

In [10]:
plt.figure(figsize=(100,80))
for i in range(burst.shape[0]):
    plt.subplot(math.ceil(burst.shape[0]/3),3,i+1)
    plt.imshow(np.fft.fftshift(weight[i,0]),'magma')
    plt.colorbar()
    plt.xticks(visible = False)
    plt.yticks(visible = False)
plt.show()

# Post processing

In [11]:
def gaussian_sharpen(Input, radius, c):
    # Input : image
    # radius : radius of H
    # c : images sharpening constant
    img = Input.copy()
    centx = int(Input.shape[0]/2)
    centy = int(Input.shape[1]/2)
    print("max max radius: "+str(math.sqrt(centx**2+centy**2))+" min max radius: "+str(min(centx, centy)))
    img = np.moveaxis(img, 2, 0) # (R, C, channel) -> (channel, R, C)
    img_result = np.zeros(img.shape)
#     print(img.shape)
    for channel in range(img.shape[0]):
        img_fft = np.fft.fft2(img[channel])
        img_fft = np.fft.fftshift(img_fft)
        for i in range(img_fft.shape[0]):
            for j in range(img_fft.shape[1]):
                cur_rad = (i-centx)**2+(j-centy)**2
                scale = math.exp(-cur_rad / (2*(radius**2)) )
                img_fft[i,j] = scale * img_fft[i,j]
        img_result[channel] = np.real(np.fft.ifft2(np.fft.ifftshift(img_fft)))
    img_result = np.moveaxis(img_result, 0, 2)
#     print(img_result.shape)
    img_result = np.where(img_result < 0, 0, img_result)
    img_result = np.where(img_result > 255, 255, img_result).astype(np.uint8)
    # image sharpening
    img_result = (c/(2*c-1))*Input - ((1-c)/(2*c-1))*img_result
    return img_result.astype(np.uint8)

### attempt to do some post-processing

In [12]:
def unsharp_masking(img, c=3/5):
    # img_median = cv2.medianBlur(img, 1)
    # img_lap = cv2.Laplacian(img, cv2.CV_64F)
    # img_sharp = img_median - 0.7*img_lap
    blur_img = cv2.GaussianBlur(img, (5,5), 0)
    img_result = c/(2*c-1)*img - (1-c)/(2*c-1)*blur_img
    img_result = np.where(img_result < 0, 0, img_result)
    img_result = np.where(img_result > 255, 255, img_result).astype(np.uint8)
    return img_result

# do unsharp masking in spatial domain
img_unsharp_masking = unsharp_masking(image_restored, 5/6)
cv2.imwrite('result_post_unsharp_masking.png', img_unsharp_masking)

# do non-local means
img_non_local_means = cv2.fastNlMeansDenoisingColored(image_restored, None, 5, 5, 7, 21)
cv2.imwrite('result_post_non_local_denoising.png', img_non_local_means)

# higt-pass filter with different filter
kernel = np.array([[0, -1, 0],
                   [-1, 5,-1],
                   [0, -1, 0]]) 
img_high_pass = cv2.filter2D(image_restored, -1, kernel)             
cv2.imwrite('result_post_high_pass.png', img_high_pass)

True

In [13]:
# input: img_restored
# output: img_restored
plt.figure(figsize=(100,80))
plt.subplot(2,1,1)
plt.imshow(cv2.cvtColor(image_restored, cv2.COLOR_BGR2RGB))
# post processing (image sharpening)
# image_sharpen = gaussian_sharpen(image_restored, 500, 0.7)
kernel = np.array([[-1, -1, -1],
                   [-1, 9,-1],
                   [-1, -1, -1]])
image_sharpen = cv2.filter2D(src=image_restored, ddepth=-1, kernel=kernel)

plt.subplot(2,1,2)
plt.imshow(cv2.cvtColor(image_sharpen, cv2.COLOR_BGR2RGB))
plt.show()

# save image

In [14]:
cv2.imwrite('result.png', image_restored)
cv2.imwrite('result_sharpen.png', image_sharpen)
cv2.imwrite('result2.png', b)
plt.figure(figsize=(100,80))
plt.subplot(3,1,1)
plt.imshow(cv2.cvtColor(image_restored, cv2.COLOR_BGR2RGB))
plt.subplot(3,1,2)
plt.imshow(cv2.cvtColor(image_sharpen, cv2.COLOR_BGR2RGB))
plt.subplot(3,1,3)
plt.imshow(cv2.cvtColor(b, cv2.COLOR_BGR2RGB))
plt.show()

# Fourier spectrums

In [15]:
spectrum=np.fft.fftshift(np.fft.fft2(cv2.imread('../images/bookshelf/result/out_fba.jpg', 0)))

t=np.abs(spectrum)
plt.imshow(np.log10(np.abs(spectrum)),'magma')
plt.colorbar()
plt.xticks(visible = False)
plt.yticks(visible = False)
plt.title('output image')
plt.show()

In [16]:
spectrum=np.fft.fftshift(np.fft.fft2(cv2.cvtColor(image_restored, cv2.COLOR_BGR2GRAY)))

t=np.abs(spectrum)
plt.imshow(np.log10(np.abs(spectrum)),'magma')
plt.colorbar()
plt.xticks(visible = False)
plt.yticks(visible = False)
plt.title('output image')
plt.show()